<a href="https://colab.research.google.com/github/gabrielborja/parc_de_montjuic/blob/main/quarterly_survey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Survey data preparation

## Importing libraries

In [1]:
# Importing python libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Loading Data

In [ ]:
# Loading data from local drive
from google.colab import files
uploaded1 = files.upload()

In [3]:
# Storing loaded data to a pandas dataframe
import io
df1 = pd.read_excel((io.BytesIO(uploaded1['survey_Q2.xlsx'])))

In [ ]:
# Checking the dataframe
df1.head(2)

In [ ]:
df1.info()

## Data cleaning

In [18]:
# Converting timestamps to datetime
df1 = df1.assign(mh_context = pd.to_datetime(df1['mh_context']),
                 answered_timestamp = pd.to_datetime(df1['answered_timestamp']))

In [ ]:
df1.info()

In [ ]:
df1['Q5'].value_counts()

## Exporting results to local drive

In [ ]:
# Exporting files to excel
with pd.ExcelWriter('melted_survey.xlsx', engine='openpyxl') as writer:
  df2_melted.to_excel(writer, sheet_name='01_melted_survey', index=False)
  df2_col.to_excel(writer, sheet_name='02_questions', index=True)
files.download('melted_survey.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Data Visualization

In [ ]:
# Importing libraries
import plotly.express as px
from ipywidgets import interact

In [ ]:
# Extracting questions for analysis
df2_questions = df2_melted['question'].unique()

In [ ]:
# Plotting histogram of survey data with matplotlib
df2_melted[df2_melted['question'] == 'q2']['answer'].plot(kind='hist')

In [ ]:
# Plotting histogram of survey data with plotly express
@interact(Questions = df2_questions)
def plot_melted_survey(Questions):
  df = df2_melted[df2_melted['question']==Questions].copy()
  fig_2a = px.histogram(df, x='answer', title=f'{Questions}: {df2_col.loc[Questions]["question"]}',
                        text_auto=True, width=800, height=500)
  fig_2a.update_layout(title_font_size=12)
  fig_2a.show()

interactive(children=(Dropdown(description='Questions', options=('q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9…